In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

asan_html=urlopen("https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomSubmain.do?partId=B000006")
bsObject=BeautifulSoup(asan_html, "html.parser")

In [2]:
# symptom link의 상세 정보
symptom_link_data=bsObject.body.find("div", {"class":"searchCont"}).find_all('a')

In [3]:
# 증상 링크
symptom_def_urls=[]
for i in range(len(symptom_link_data)):
    link=symptom_link_data[i].get('href')
    symptom_def_urls.append('https://www.amc.seoul.kr'+link)
print(symptom_def_urls)

['https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000779', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000258', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000255', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000259', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000263', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000260', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000261', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId=SS000262']


In [4]:
# 증상 이름 
symptom_names=[]
for i in range(len(symptom_link_data)):
    name=symptom_link_data[i].get_text()
    symptom_names.append(name)
print(symptom_names)

['관절통', '굽은 등', '뼈의 기형', '요통', '자세이상', '척추 측만', '척추 후만', '척추와 허리 디스크']


In [5]:
# 증상 id
symptom_ids=[]
for i in range(len(symptom_link_data)):
    id_name="cont1_"+str(i+1)
    symptom_id=bsObject.body.find("input", {'id':id_name}).get('value')
    symptom_ids.append(symptom_id)
print(symptom_ids)

['SS000779', 'SS000258', 'SS000255', 'SS000259', 'SS000263', 'SS000260', 'SS000261', 'SS000262']


In [6]:
back_symptoms = dict(zip(symptom_ids, symptom_names))
print('증상 수:',len(back_symptoms))
print(back_symptoms)

증상 수: 8
{'SS000779': '관절통', 'SS000258': '굽은 등', 'SS000255': '뼈의 기형', 'SS000259': '요통', 'SS000263': '자세이상', 'SS000260': '척추 측만', 'SS000261': '척추 후만', 'SS000262': '척추와 허리 디스크'}


In [7]:
# 불필요 데이터 pop

In [8]:
# 각 증상 선택 결과 link
res_links=[]
for idx in back_symptoms:
    res_link='https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds='+idx
    res_links.append(res_link)
print(res_links)
print("개수: ", len(res_links))

['https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000779', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000258', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000255', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000259', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000263', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000260', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000261', 'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds=SS000262']
개수:  8


In [9]:
"""
"""

idx_datas=['SS000779', 'SS000258', 'SS000255', 'SS000259', 
           'SS000263', 'SS000260', 'SS000261', 'SS000262']
res_links=[]
all_needed_index=[]
for idx in idx_datas:
    res_link='https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomList.do?searchFrmPartId=B000006&symptomIds='+idx
    res_links.append(res_link)
# print(res_links)
print("개수: ", len(res_links))

개수:  8


In [10]:
import re

i=0
detail_info_list=[]   # 세부 정보
for idx in idx_datas:
    sympt_res_html=urlopen(res_links[i])     # 추린 내용
    sympt_res_Object=BeautifulSoup(sympt_res_html, "html.parser")
    print('선택된 증상 이름: ', back_symptoms[idx])   
    
    each_symp_data=[]     # 증상별 data
    res_names=[]
    symptom_res_link_data=sympt_res_Object.body.find_all("strong", {"class":"contTitle"})

    for j in range(len(symptom_res_link_data)):
        one_disease_data=[]
        
        res_names.append(symptom_res_link_data[j].get_text().strip())  # 예상 질병 이름 모으기
        one_disease_data.append({"질병 이름":symptom_res_link_data[j].get_text().strip()})
        
        contentId=symptom_res_link_data[j].find('a').get('href')
        res_link_symp='https://www.amc.seoul.kr'+contentId
        sympt_res_html2=urlopen(res_link_symp)     # 추린 내용

        sympt_res_Object2=BeautifulSoup(sympt_res_html2, "html.parser")
        
        # 진료과 관련 정보
        department_datas=sympt_res_Object2.body.find("div", {"class":"contBox"}).find_all('dd')
        department_idx_datas=sympt_res_Object2.body.find("div", {"class":"contBox"}).find_all('dt')
        
        # 진료과
        for depart_idx in range(len(department_datas)):
            if department_idx_datas[depart_idx].get_text().strip()=="진료과":
                dapartment_name=department_datas[depart_idx].get_text().replace(",", "")
                dapartment_name_str=re.findall(r'\S+', dapartment_name)
                one_disease_data.append({"진료과":dapartment_name_str})
                
        details_names=sympt_res_Object2.body.find("div", {"class":"contDescription"}).find_all('dt')
        details_datas=sympt_res_Object2.body.find("div", {"class":"contDescription"}).find_all('dd')
   
        k=0
        for data in details_names:
            if data.get_text().strip()=="정의":
                defi=details_datas[k].find('p')
                try: one_disease_data.append({"정의":defi.get_text().strip()})
                except AttributeError:  print("Attribute Error!")
                
            elif data.get_text().strip()=="원인":
                cause=details_datas[k].find('p')
                try: one_disease_data.append({"원인":cause.get_text().strip()})
                except AttributeError:  print("Attribute Error!")
                
            elif data.get_text().strip()=="증상":
                sympt=details_datas[k].find('p')
                try:  one_disease_data.append({"증상":sympt.get_text().strip()})
                except AttributeError: print("Attribute Error!")
            
            elif data.get_text().strip()=="치료":
                cure=details_datas[k].find('p')
                try:  one_disease_data.append({"치료":cure.get_text().strip()})
                except AttributeError:  print("Attribute Error!")
            k=k+1
        # print(one_disease_data)
        each_symp_data.append(one_disease_data)
   
    # print()
    detail_info_list.append(each_symp_data)
    # print(res_names)
    i=i+1
    # print()

선택된 증상 이름:  관절통
선택된 증상 이름:  굽은 등
선택된 증상 이름:  뼈의 기형
선택된 증상 이름:  요통
선택된 증상 이름:  자세이상
선택된 증상 이름:  척추 측만
선택된 증상 이름:  척추 후만
선택된 증상 이름:  척추와 허리 디스크


In [11]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

cred = credentials.Certificate("whereigo-f28da-firebase-adminsdk-b1ky0-b95a286f3a.json")
firebase_admin.initialize_app(cred, {
    'databaseURL':'https://whereigo-f28da-default-rtdb.firebaseio.com/:'  # database 주소 입력
})

In [12]:
i=0
for idx in idx_datas:
    sympt_res_html=urlopen(res_links[i])     # 추린 내용
    sympt_res_Object=BeautifulSoup(sympt_res_html, "html.parser")
    
    # symptom result link의 상세 정보
    symptom_res_link_data=sympt_res_Object.body.find_all("strong", {"class":"contTitle"})
    print('선택된 증상 이름: ', back_symptoms[idx])   

    for j in range(len(symptom_res_link_data)):
        dir=db.reference("등/"+back_symptoms[idx]+"/"+symptom_res_link_data[j].get_text().strip())
        print('예상 질병 이름: ',symptom_res_link_data[j].get_text().strip())
        print("len(detail_info_list[i][j]): ", len(detail_info_list[i][j]))
        print("예상 질병 이름 길이", len(detail_info_list[i][j])) 
        
        for k in range(len(detail_info_list[i][j])):
            dir.update(detail_info_list[i][j][k])
            print(detail_info_list[i][j][k], "\n")
        print("\n\n")
    i=i+1

선택된 증상 이름:  관절통
예상 질병 이름:  간염(Hepatitis)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '간염(Hepatitis)'} 

{'진료과': ['소화기내과']} 

{'정의': '간염은 간세포 조직에 염증이 생긴 것을 의미합니다.'} 

{'원인': '간염은 바이러스, 약물, 알코올, 화학 약물, 독초 등으로 인하여 발병합니다. 바이러스성 간염은 원인 병원체에 따라 A형, B형, C형, D형, E형, G형으로 구분됩니다. 드물기는 하지만 자가 면역성 간염이나 윌슨병 등도 간염의 원인이 될 수 있습니다.'} 

{'증상': '간염은 그 지속 기간에 따라서 급성(6개월 이내)과 만성(6개월 이상)으로 구분됩니다.'} 

{'치료': '급성 간염 환자들은 대부분 특별한 치료 없이 충분한 휴식과 영양 섭취만으로도 회복됩니다. 하지만 B형, C형 간염 바이러스로 인한 급성 간염 환자들은 간혹 급성 간부전으로 위험해지거나 만성 간염으로 진행할 수 있으므로 항바이러스 치료가 필요하기도 합니다. 자가면역성 간염이나 윌슨병의 경우 심한 상태에서도 특이적인 치료약으로 회복되는 경우가 많으므로 반드시 전문적인 치료가 필요합니다. 간염이 급성 간부전으로 진행한 경우에는 중환자실 치료가 필요할 수 있으며 간 이식을 고려해야 할 수도 있습니다. 만성 간염의 경우 원인에 따른 특이적인 치료법이 존재하므로 전문의와 상담해야 합니다.'} 




예상 질병 이름:  강직성 척추염(Ankylosing spondylitis)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '강직성 척추염(Ankylosing spondylitis)'} 

{'진료과': ['재활의학과', '류마티스내과']} 

{'정의': '강직성 척추염은 척추의 주 병변이 특징인 만성 관절염의 일종입니다. 강직성에서 ‘강직’은 ‘뻣뻣해짐’ 또는 ‘굳는 것’을 의미하고, 척추염은 말 그대로 ‘척추에

{'증상': '독성 간염에 걸리면 일반적인 급성 간염의 증상이 모두 나타날 수 있습니다. 증상이 없는 경우도 있으며, 바이러스 간염의 증상과 같은 오심, 구토, 식욕 부진, 피곤함, 황달, 오른쪽 상복부 통증이 발생하기도 합니다. 원인 물질에 따라 특징적인 증상이 나타나기도 합니다. 특이한 체질에 의한 독성 간염의 경우 피부 발진, 관절통, 발열 등의 증상이 발생할 수 있습니다. 진통제인 아세트아미노펜을 과다하게 복용한 경우 수 시간 만에 오심, 구토, 설사, 복통 등이 발생합니다. 수일 후에는 간 손상에 의한 황달, 복수, 간성뇌증 등의 증상이 나타날 수 있습니다.'} 

{'치료': '독성 간염의 치료 방법으로는 원인 약물의 복용을 중단하는 것이 가장 중요합니다. 대부분 약물의 복용을 중단하면 간 기능이 정상으로 회복됩니다. 심한 간 손상에 의해 간성뇌증이 발생하면 환자의 의식은 나빠집니다. 이것이 급성 간부전입니다. 급성 간부전으로 진행하는 경우 예후가 매우 불량하므로 응급 간 이식이 필요할 수 있습니다.'} 




예상 질병 이름:  라임병(Lyme disease)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '라임병(Lyme disease)'} 

{'진료과': ['감염내과', '피부과']} 

{'정의': '라임병은 곤충인 진드기가 사람을 무는 과정에서 보렐리아균이 신체에 침범하여 여러 기관에 병을 일으키는 감염성 질환을 의미합니다. 라임병은 미국 북동부 지역의 풍토병입니다. 미세한 진드기가 동물의 피를 빨아먹으면, 그 박테리아는 진드기의 내장에서 여생을 보냅니다. 이 진드기가 동물이나 사람을 물 때 박테리아가 혈류로 유입되어 발병합니다.'} 

{'원인': '라임병의 원인은 진드기 속에서 사는 나선균입니다. 진드기는 주로 동물, 특히 사슴이나 작은 설치류의 몸에 붙어 다닙니다. 사람이 보렐리아균에 감염된 진드기에 물리면 라임병에 걸립니다.'} 

{'증상': '라임병은 보통 3단계로 진행됩니

{'치료': '어떤 아밀로이드증도 그 자체를 치료하는 방법은 없습니다. 아밀로이드증을 완치하기는 어렵지만, 아밀로이드 단백 생성을 억제하는 치료 방법을 통해 증상을 완화시키고 생명을 연장시킬 수 있습니다.'} 




선택된 증상 이름:  굽은 등
예상 질병 이름:  VDT 증후군(Visual Display Terminal Syndrome)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': 'VDT 증후군(Visual Display Terminal Syndrome)'} 

{'진료과': ['정형외과']} 

{'정의': 'VDT 증후군이란 장시간 동안 모니터를 보며 키보드를 두드리는 작업을 할 때 생기는 각종 신체적, 정신적 장애를 이르는 말입니다. 이는 장시간 동안 컴퓨터, 스마트폰, 모바일 디바이스 등을 보는 젊은이에게 많이 나타나는 질환입니다.'} 

{'원인': 'VDT 증후군을 유발하는 요인은 다음과 같습니다.'} 

{'증상': '컴퓨터나 단말기를 자주 사용하면서 다음의 세 가지 증상 중 한 가지 증상이 나타난다면 VDT 증후군을 의심할 수 있습니다.'} 

{'치료': 'VDT 증후군의 가장 효과적인 치료 방법은 체조입니다. 컴퓨터 사용자에게는 손목, 목, 어깨 위주의 체조가 좋습니다. 주로 의자에 앉아서 할 수 있는 체조가 도움이 됩니다. 체조 방법은 8~12초간 자세를 유지하고, 3~5번 반복하는 것입니다. 스트레칭을 하는 동안은 자연스럽게 호흡합니다. 이러한 체조를 하루에 여러 번 시간이 날 때마다 실시합니다.'} 




예상 질병 이름:  구루병(Rickets)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '구루병(Rickets)'} 

{'진료과': ['정형외과', '소아정형외과']} 

{'정의': '구루병은 칼슘과 인의 대사 장애로 인해 뼈 발육에 장애가 발생하는 질환을 의미합니다. 흉곽 모양이나 척추, 다리의 변형을 동반합니다.

{'치료': '난소암의 기본적인 치료 방법은 수술로 종양을 최대한 제거하고 항암제를 투여하는 것입니다. 환자가 미혼이거나 아기를 낳아야 하는데 암이 난소에만 국한되었다면 수술로 한쪽 난소만 제거하고 자궁과 반대쪽 난소를 남기기도 합니다. 하지만 난소암 환자들은 대부분 수술로 자궁, 양쪽 난소, 맹장, 대망 등 여러 부위를 제거합니다. 수술로 눈에 보이는 종양을 다 제거하여 남은 종양이 없거나 그 크기가 1cm 이하라면 좋은 결과를 기대할 수 있습니다. 그 이후 암의 단계에 따라 추가적으로 항암 치료를 진행합니다.'} 




예상 질병 이름:  난소의 양성 종양(Benign ovary tumor)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '난소의 양성 종양(Benign ovary tumor)'} 

{'진료과': ['산부인과']} 

{'정의': '난소의 양성 종양은 난소에서 악성(암)이 아닌 양성 신생물이 발생하는 질환을 의미합니다.'} 

{'원인': '난소의 양성 종양이 발생하는 원인은 아직 정확하게 밝혀지지 않았습니다. 지속적인 배란이나 외부의 자극이 낭종의 발생을 유발한다는 가설이 유력합니다. 지속적인 배란으로 난소 표면에 상처가 생기면 난소 상피세포가 난소 내부로 함몰되어 들어갑니다. 그러면 이 속에 액체가 고이고 유출되지 않아 점차 커지면서 낭종이 발생합니다.'} 

{'증상': '난소 양성 종양의 증상으로는 하복부 통증, 요통, 빈번한 소변, 복부 팽만감, 배변 장애가 있습니다. 낭종이 꼬일 경우 심한 복통이 발생하기도 하지만, 아무런 증상이 나타나지 않을 수도 있습니다.'} 

{'치료': '난소 양성 종양의 치료 여부는 증상의 유무, 환자의 연령, 종양의 크기, 악성의 가능성 등과 같은 여러 요소를 고려하여 결정합니다.'} 




예상 질병 이름:  노인성 척추후만증(Senile Kyphosis)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'

{'진료과': ['정형외과']} 

{'정의': '요족은 발등이 정상보다 높이 올라오는 상태를 의미합니다. 즉, 발바닥의 아치가 높아서 옆에서 보면 발바닥이 위로 볼록하게 올라간 상태를 말합니다. 아치가 높아 체중이 발바닥 골고루 분산되지 못하고 발뒤꿈치와 발 앞쪽에 쏠려 여러 가지 증상이 나타납니다.'} 

{'원인': '요족은 보통 신경 계통의 질환과 동반되어 나타납니다. 예전에는 요족의 20% 정도만 원인을 알 수 있었습니다. 그러나 현재는 다양하게 발전한 검사 방법으로 80% 이상에서 원인을 알 수 있습니다.'} 

{'증상': '요족 환자는 발바닥이 높아서 걸을 때 발뒤꿈치의 충격이 그대로 발의 앞쪽으로 전달됩니다. 이로 인해 발바닥 앞쪽의 통증이 심하고 굳은살이 생깁니다. 이것은 보행에 장애가 되고, 허리나 척추에 무리가 되어 허리 통증을 유발합니다. 또한 발가락이 구부러지는 갈퀴족지가 나타나고, 발목이 잘 삡니다. 요족을 치료하지 않으면 발의 모양이 변형될 뿐만 아니라, 점차 종아리 근육이 경직되어 발 전체에 심한 통증과 변형이 올 수 있습니다. 따라서 적절한 치료가 필요합니다.'} 

{'치료': '통증이 없고 일상생활에 불편이 없는 정도라면 보존적인 방법으로 치료합니다. 담당 의사와 상의한 후 중족골(발등뼈) 패드를 착용하거나 보조 신발을 신을 수 있습니다. 하지만 통증이 심하면 수술적 치료를 고려해야 합니다. 요족이 생긴 원인에 따라, 또 증상의 경중에 따라 수술 방법이 달라집니다.'} 




예상 질병 이름:  요추 전방전위증(Lumbar spondylolisthesis)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '요추 전방전위증(Lumbar spondylolisthesis)'} 

{'진료과': ['신경외과', '정형외과']} 

{'정의': '요추 전방전위증은 하나의 척추가 여기에 인접하는 밑의 척추에 비해 정상적인 정렬을 이루지 못하고 앞으로 빠져 있는 상태를 말합니다. 요추

{'원인': '22번 염색체 미세결실 증후군은 22번 염색체의 장완 근위부(22q11.2)의 미세결실로 인해 나타납니다. 이 부위의 결실은 제 3, 4 인두낭의 정상 발육에 영향을 끼쳐 흉선, 부갑상선, 대동맥궁 등의 심장 발달 및 여러 선천성 기형을 유발합니다. 22번 염색체 장완의 미세결실은 자연발생적으로 일어나는 경우가 많지만, 10~15%는 가족력에 의해 나타납니다. 즉, 부모 중 한 사람이 결실된 22번 염색체를 가지고 있는 경우, 자녀가 22번 염색체 미세결실 증후군을 앓게 될 확률은 50%입니다.'} 

{'증상': '22번 염색체 미세결실 증후군은 심장 기형, 구개열(구순열), 비음, 안면 기형(긴 코, 돌출된 콧대, 짧은 인중, 작은 턱에 편평한 광대뼈, 좁은 안검열, 양안 격리증, 변형 귀, 작은 입), 저칼슘혈증을 유발하며, 때로 발작을 일으키는 부갑상선기능저하증, 갑상선기능저하, T 림프구와 관련된 면역 체계 이상, 잦은 상기도 감염, 잦은 중이염, 성장 발달 지연, 위장관 기형, 신장, 비뇨기계 기형, 행동 장애 및 학습 장애, 정신과적 이상, 홍채 결손이나 원시와 같은 시력 장애, 청각 장애(유스타키오관의 기능 이상, 중이염, 청력 상실), 소두증, 작은 키, 작은 몸통, 척추 측만증, 앞으로 돌출된 작은 어깨, 발 기형, 잠복 고환 등이 동반됩니다.'} 

{'치료': '22번 염색체 미세결실 증후군의 근본적인 치료법은 없습니다. 그러나 증상에 따라 수술, 언어 치료, 물리 치료, 정기적인 치료, 특수 교육, 정신과적 상담을 시행할 수 있습니다.'} 




예상 질병 이름:  근육병(Myopathy)
len(detail_info_list[i][j]):  6
예상 질병 이름 길이 6
{'질병 이름': '근육병(Myopathy)'} 

{'진료과': ['소아재활의학과']} 

{'정의': '근육병은 점진적인 근력 감소로 인해 보행 능력 상실, 호흡 근력 약화, 심장 기능 약화 등이 나타나는 진행성 질환입니다. 결국에는 신체 장애가 생겨

{'치료': '무지외반증의 치료는 환자가 느끼는 증상, 변형의 심각성, 치료의 목적(염증과 통증의 조절인지, 미용적 문제까지 해결하는 것인지) 등에 따라 달라집니다. 무지외반증이 심하지 않은 경우, 증상을 조절하고 변형의 진행을 늦추기 위해서 볼이 넓고 부드러운 신발을 착용하거나, 교정을 위한 깔창이나 보형물을 사용합니다. 심한 변형이 동반된 경우, 수술을 통해 교정해야 합니다. 수술을 통해 돌출 부위의 뼈를 깎고 인대와 연부 조직의 길이를 조절합니다.'} 




